Here just for simplicity, I write all preprocess code together. If you are instested what happend in the preprocess step, please move to this [notebook](https://github.com/BrambleXu/nlp-beginner-guide-keras/blob/master/char-level-cnn/notebooks/char-level-text-preprocess-with-keras-summary.ipynb). 

In [2]:
# write all code in one cell 

#========================Load data=========================
import numpy as np
import pandas as pd

train_data_source = '../data/ag_news_csv/train.csv'
test_data_source = '../data/ag_news_csv/test.csv'

train_df = pd.read_csv(train_data_source, header=None)
test_df = pd.read_csv(test_data_source, header=None)

# concatenate column 1 and column 2 as one text
for df in [train_df, test_df]:
    df[1] = df[1] + df[2]
    df = df.drop([2], axis=1)
    
# convert string to lower case 
train_texts = train_df[1].values 
train_texts = [s.lower() for s in train_texts] 

test_texts = test_df[1].values 
test_texts = [s.lower() for s in test_texts] 

#=======================Convert string to index================
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(train_texts)
# If we already have a character list, then replace the tk.word_index
# If not, just skip below part

#-----------------------Skip part start--------------------------
# construct a new vocabulary 
alphabet="abcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1
    
# Use char_dict to replace the tk.word_index
tk.word_index = char_dict.copy() 
# Add 'UNK' to the vocabulary 
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1
#-----------------------Skip part end----------------------------

# Convert string to index 
train_sequences = tk.texts_to_sequences(train_texts)
test_texts = tk.texts_to_sequences(test_texts)

# Padding
train_data = pad_sequences(train_sequences, maxlen=1014, padding='post')
test_data = pad_sequences(test_texts, maxlen=1014, padding='post')

# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')

#=======================Get classes================
train_classes = train_df[0].values
train_class_list = [x-1 for x in train_classes]

test_classes = test_df[0].values
test_class_list = [x-1 for x in test_classes]

from keras.utils import to_categorical
train_classes = to_categorical(train_class_list)
test_classes = to_categorical(test_class_list)

Using TensorFlow backend.


## Construct Model

We implement the char_cnn_zhang model from this paper:

- Xiang Zhang, Junbo Zhao, Yann LeCun. [Character-level Convolutional Networks for Text Classification](http://arxiv.org/abs/1509.01626). NIPS 2015

The model structure:

![](https://cdn-images-1.medium.com/max/1600/0*fovAEUSdSkbsnJw5.png)

This graph may look difficult to understand. Here is the model setup. 


![](https://img-blog.csdn.net/20170721104727009)


If you want to see the detail for this model, please move to this [notebook](https://github.com/BrambleXu/nlp-beginner-guide-keras/blob/master/char-level-cnn/notebooks/char_cnn_zhang.ipynb)

We choose the small frame, 256 filters in convolutional layer and 1024 output units in dense layer. 

- Embedding Layer
- Six convolutional layers, and 3 convolutional layers followed by a max pooling layer
- Two fully connected layer(dense layer in keras), neuron units are 1024. 
- Output layer(dense layer), neuron units depends on classes. In this task, we set it 4. 

First we have to construct a embedding index. Beside the 68 characters in `alphabet`, we add `UNK` with `index 69`. These 69 characters are saved to `tk.word_index`, we could output it to see:

In [3]:
print(tk.word_index)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '0': 27, '1': 28, '2': 29, '3': 30, '4': 31, '5': 32, '6': 33, '7': 34, '8': 35, '9': 36, ',': 37, ';': 38, '.': 39, '!': 40, '?': 41, ':': 42, "'": 43, '"': 44, '/': 45, '\\': 46, '|': 47, '_': 48, '@': 49, '#': 50, '$': 51, '%': 52, '^': 53, '&': 54, '*': 55, '~': 56, '`': 57, '+': 58, '-': 59, '=': 60, '<': 61, '>': 62, '(': 63, ')': 64, '[': 65, ']': 66, '{': 67, '}': 68, 'UNK': 69}


In [4]:
vocab_size = len(tk.word_index)
vocab_size

69

We can use one-hot vector to represent these 69 words. Because Keras use `0` for `PAD`. We add first line containing all 0 to represent the `PAD`

In [5]:
embedding_weights = [] #(70, 69)
embedding_weights.append(np.zeros(vocab_size)) #(0, 69)

for char, i in tk.word_index.items(): # from index 1 to 69
    onehot = np.zeros(vocab_size)
    onehot[i-1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)

In [6]:
print(embedding_weights.shape) # first row all 0 for PAD, 68 char, last row for UNK
embedding_weights

(70, 69)


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [7]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

In [8]:
# parameter 
input_size = 1014
# vocab_size = 69
embedding_size = 69
conv_layers = [[256, 7, 3], 
               [256, 7, 3], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, 3]]

fully_connected_layers = [1024, 1024]
num_of_classes = 4
dropout_p = 0.5
optimizer = 'adam'
loss = 'categorical_crossentropy'

In [9]:
# Embedding layer Initialization
embedding_layer = Embedding(vocab_size+1, 
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

In [19]:
# Model 

# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64')  # shape=(?, 1014)
# Embedding 
x = embedding_layer(inputs)
# Conv 
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x) 
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x) # Final shape=(None, 34, 256)
x = Flatten()(x) # (None, 8704)
# Fully connected layers 
for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x) # dense_size == 1024
    x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(num_of_classes, activation='softmax')(x)
# Build model
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy']) # Adam, categorical_crossentropy
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1014)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1014, 69)          4830      
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 1008, 256)         123904    
_________________________________________________________________
activation_13 (Activation)   (None, 1008, 256)         0         
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 336, 256)          0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 330, 256)          459008    
_________________________________________________________________
activation_14 (Activation)   (None, 330, 256)          0         
__________

## train the model
Because here I just use CPU to run the model, so I only use 10000 samples for trianing and 1000 samples for testing.

In [11]:
# 1000 training samples and 100 testing samples
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)

x_train = train_data[indices][:1000]
y_train = train_classes[indices][:1000]

x_test = test_data[:100]
y_test = test_classes[:100]

In [13]:
# Training
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=128,
          epochs=10,
          verbose=2)

Train on 1000 samples, validate on 100 samples
Epoch 1/10
 - 72s - loss: 1.4544 - val_loss: 1.3411
Epoch 2/10
 - 68s - loss: 1.3877 - val_loss: 1.3666
Epoch 3/10
 - 61s - loss: 1.3798 - val_loss: 1.3100
Epoch 4/10
 - 60s - loss: 1.3666 - val_loss: 1.2561
Epoch 5/10
 - 104s - loss: 1.3640 - val_loss: 1.2740
Epoch 6/10
 - 77s - loss: 1.3511 - val_loss: 1.3082
Epoch 7/10
 - 80s - loss: 1.3347 - val_loss: 1.2682
Epoch 8/10
 - 81s - loss: 1.2980 - val_loss: 1.2315
Epoch 9/10
 - 78s - loss: 1.1978 - val_loss: 1.3046
Epoch 10/10
 - 100s - loss: 0.9743 - val_loss: 1.3859


Because we use a small dataset, so the model is easy to overfit. 